In [1]:
import pandas as pd
import numpy as np

In [2]:
with open('Korean_movie_reviews_2016.txt', encoding='utf-8') as f:
    docs = [doc.strip().split('\t') for doc in f]
    docs = [(doc[0], int(doc[1])) for doc in docs if len(doc) == 2]
    # To read the second and third column info from each row
    texts, labels = zip(*docs)
    # 둘을 분리해서 별도의 list 변수로 저장

In [3]:
len(labels)

165384

In [4]:
sum(labels)/len(labels)

0.5248754413969913

In [5]:
# To split the data into training and test datasets
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.1, random_state=0)

In [6]:
train_texts[0]

'제목 같은 영화 점 정도 평점 밸런스 위해 볼 함'

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
tf_vectorizer = CountVectorizer() 
tf_train_features = tf_vectorizer.fit_transform(train_texts) 
tf_test_features = tf_vectorizer.transform(test_texts)

In [9]:
vocablist = [word for word, _ in sorted(tf_vectorizer.vocabulary_.items(), key=lambda x:x[1])]

In [10]:
len(vocablist)

47707

In [11]:
vocablist[:10]

['가가', '가가호호', '가각', '가감', '가거', '가겄', '가게', '가겠', '가격', '가계']

In [12]:
from sklearn.linear_model import LogisticRegression

### TF 정보 사용해 보기

In [13]:
# tf matrix를 사용한 경우
lr_tf = LogisticRegression(max_iter=1000) 
lr_tf.fit(tf_train_features, train_labels) # 학습
pred_labels = lr_tf.predict(tf_test_features)

In [14]:
from sklearn.metrics import accuracy_score
print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

Misclassified samples: 1837 out of 16539
Accuracy: 0.89


In [15]:
review_index = 6
print(test_texts[review_index])
print("The predicted value: {} and probability: {}".format(lr_tf.predict(tf_test_features[review_index]), lr_tf.predict_proba(tf_test_features[review_index])))

제작 진 방패 놈 안티 듯 방패 놈 친구 하려 베프 정돈 돼 함 있어 물건 너 갔 오로지 베프 편 들 소통 설득 안 하는 막장 답답 캐릭터 베프키따 뇌 조종 받고 하이드 갈 놈 막판씬 진심 발암 평점 점 정도 적당
The predicted value: [0] and probability: [[9.99835518e-01 1.64481511e-04]]


In [16]:
# Get coefficients of the model 
coefficients = lr_tf.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)
# 학습에 사용된 각 단어마다의 coefficient (즉 weight) 값이 존재
# coefficient값이 큰 순으로 정렬 'reverse=True'

print(sorted_coefficients[:5])
# print top 50 positive words
for word, coef in sorted_coefficients[:50]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))
# print top 50 negative words
for word, coef in sorted_coefficients[-50:]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))

[(37124, 3.772976720594885), (5803, 3.635302495056776), (35215, 3.1356690689998867), (45663, 2.936982718495355), (34949, 2.9361252013373402)]
존잼 (3.773)
꿀잼 (3.635)
재밌던 (3.136)
핵꿀잼 (2.937)
재미있게 (2.936)
개꿀잼 (2.918)
졸잼 (2.848)
여운 (2.815)
재밌구 (2.796)
재미있었 (2.787)
재밌었 (2.735)
굿굿 (2.710)
재밌게 (2.683)
재미있네 (2.675)
재밌고 (2.645)
만족스러웠 (2.643)
완벽하다 (2.571)
깔끔한 (2.513)
만족합 (2.506)
완벽했 (2.482)
사랑합 (2.479)
강추 (2.469)
완벽한 (2.458)
재밌네 (2.442)
재미있어 (2.441)
사랑해 (2.402)
따뜻해 (2.401)
재밌어 (2.394)
최고다 (2.375)
낮아 (2.363)
행복했 (2.351)
흥미진진 (2.306)
짱잼 (2.296)
기대됩 (2.281)
놓칠 (2.274)
저격 (2.269)
재미있고 (2.258)
감탄 (2.255)
사랑스러운 (2.255)
재밋어 (2.252)
영화평론가 (2.251)
걱정했 (2.248)
흠잡 (2.241)
꾸르잼 (2.232)
심장 (2.223)
지루하지 (2.218)
행복해 (2.197)
보람 (2.195)
십점 (2.195)
기대되는 (2.192)
불륜 (-2.414)
별루 (-2.415)
답답해 (-2.429)
미화 (-2.432)
날조 (-2.433)
망친 (-2.450)
지루하고 (-2.453)
팔이 (-2.456)
반개 (-2.463)
오글거려 (-2.464)
나가고 (-2.466)
거품 (-2.469)
노답 (-2.477)
왜곡 (-2.479)
디워 (-2.489)
실망했 (-2.491)
망쳐 (-2.500)
엉망 (-2.509)
그닥 (-2.511)
독점 (-2.554)
이하 (-2.556)

### TF-IDF 정보 이용해 보기

In [20]:
tfidf_vectorizer = TfidfVectorizer() 
tfidf_train_features = tfidf_vectorizer.fit_transform(train_texts) 
tfidf_test_features = tfidf_vectorizer.transform(test_texts)

In [21]:
# tfidf matrix를 사용한 경우
lr_tfidf = LogisticRegression(max_iter=1000) # Lasso regression
lr_tfidf.fit(tfidf_train_features, train_labels) # 학습
pred_labels = lr_tfidf.predict(tfidf_test_features)


In [22]:
print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

Misclassified samples: 1829 out of 16539
Accuracy: 0.89


In [23]:
# Get coefficients of the model 
coefficients = lr_tfidf.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)
# 학습에 사용된 각 단어마다의 coefficient (즉 weight) 값이 존재
# coefficient값이 큰 순으로 정렬 'reverse=True'

print(sorted_coefficients[:5])
# print top 50 positive words
for word, coef in sorted_coefficients[:50]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))
# print top 50 negative words
for word, coef in sorted_coefficients[-50:]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))

[(5803, 7.377382116200149), (35234, 6.776375979162454), (35197, 6.605370152185033), (28723, 6.410074819726493), (40640, 6.231654780306138)]
꿀잼 (7.377)
재밌었 (6.776)
재밌게 (6.605)
여운 (6.410)
최고 (6.232)
재미있게 (6.101)
재미있었 (5.747)
재밌어 (5.704)
존잼 (5.471)
가슴 (5.280)
강추 (5.236)
재밌고 (5.015)
대박 (4.783)
재밌 (4.311)
개꿀잼 (4.299)
감사합 (4.288)
재밌네 (4.279)
감동 (4.228)
지루하지 (4.173)
굿굿 (4.168)
흥미진진 (4.156)
재미있네 (4.132)
심장 (4.119)
재미있어 (4.094)
탄탄 (4.037)
울었 (4.011)
가는 (3.994)
감탄 (3.980)
재밌던 (3.975)
최고다 (3.960)
완벽한 (3.935)
재미있고 (3.886)
지루할 (3.839)
졸잼 (3.802)
감사 (3.785)
사랑해 (3.756)
눈물 (3.699)
재밋어 (3.670)
빠져 (3.668)
유쾌 (3.660)
즐겁 (3.645)
멋지 (3.603)
매력 (3.599)
테러 (3.576)
저격 (3.546)
핵꿀잼 (3.510)
펑펑 (3.500)
재미있 (3.499)
전문가 (3.466)
시키 (3.389)
발연기 (-3.699)
노답 (-3.703)
전형 (-3.720)
신파극 (-3.727)
아까웠 (-3.796)
어이 (-3.800)
거품 (-3.843)
독점 (-3.869)
실망했 (-3.907)
불쾌 (-3.913)
나가고 (-4.057)
댓글알바 (-4.105)
유치 (-4.187)
아깝 (-4.189)
불면증 (-4.191)
낭비 (-4.238)
엉망 (-4.312)
불륜 (-4.348)
하품 (-4.351)
아까운 (-4.356)
지루해 (-4.365)
비추 (-4.427)
왜곡 (-4